In [1]:
import polars as pl

# 表达式

`fn(Series) -> Series`

* 懒惰评估
    - 可以被优化
    - 向库提供背景信息，并做出明智的决定
* 高度并行
* 上下文相关
    - 选择 / 投影 -> `Series` = **列、文本或值**
    - 聚合 -> `Series` = **组**


In [2]:
df = pl.DataFrame(
    {
        "A": [1, 2, 3, 4, 5],
        "fruits": ["banana", "banana", "apple", "apple", "banana"],
        "B": [5, 4, 3, 2, 1],
        "cars": ["beetle", "audi", "beetle", "beetle", "beetle"],
        "optional": [28, 300, None, 2, -30],
    }
)
df

A,fruits,B,cars,optional
i64,str,i64,str,i64
1,"""banana""",5,"""beetle""",28
2,"""banana""",4,"""audi""",300
3,"""apple""",3,"""beetle""",null
4,"""apple""",2,"""beetle""",2
5,"""banana""",1,"""beetle""",-30


# 选择上下文

In [3]:
# 我们可以通过名称来选择

(df.select([
    pl.col("A"),
    "B",      # "B"这一列将会被推理
    pl.lit("B"),  # 我们必须告诉Polars, 我们"B"的文字信息
    pl.col("fruits"),
]))

A,B,literal,fruits
i64,i64,str,str
1,5,"""B""","""banana"""
2,4,"""B""","""banana"""
3,3,"""B""","""apple"""
4,2,"""B""","""apple"""
5,1,"""B""","""banana"""


In [4]:
# 可以通过正则表达式（起始为‘^’终止为'$'）来选择列

(df.select([
    pl.col("^A|B$").sum() # 求和
]))

A,B
i64,i64
15,15


In [5]:
# 可以通过名称选取多个列

(df.select([
    pl.col(["A", "B"]).sum() # 求和
]))

A,B
i64,i64
15,15


In [6]:
# 我们可以以正常顺序选择所有事物
# 然后我们可以以逆序的方式选择事物

(df.select([
    pl.all(),
    pl.all().reverse().suffix("_reverse") # 表格中所有列逆序（并且文字中后缀为_reverse
]))

A,fruits,B,cars,optional,A_reverse,fruits_reverse,B_reverse,cars_reverse,optional_reverse
i64,str,i64,str,i64,i64,str,i64,str,i64
1,"""banana""",5,"""beetle""",28,5,"""banana""",1,"""beetle""",-30
2,"""banana""",4,"""audi""",300,4,"""apple""",2,"""beetle""",2
3,"""apple""",3,"""beetle""",null,3,"""apple""",3,"""beetle""",null
4,"""apple""",2,"""beetle""",2,2,"""banana""",4,"""audi""",300
5,"""banana""",1,"""beetle""",-30,1,"""banana""",5,"""beetle""",28


In [7]:
# 所有表达式并行地运行
# 单值'Series'被广播到'DataFrame'的形状`

(df.select([
    pl.all(),
    pl.all().sum().suffix("_sum")
]))

A,fruits,B,cars,optional,A_sum,fruits_sum,B_sum,cars_sum,optional_sum
i64,str,i64,str,i64,i64,str,i64,str,i64
1,"""banana""",5,"""beetle""",28,15,null,15,null,300
2,"""banana""",4,"""audi""",300,15,null,15,null,300
3,"""apple""",3,"""beetle""",null,15,null,15,null,300
4,"""apple""",2,"""beetle""",2,15,null,15,null,300
5,"""banana""",1,"""beetle""",-30,15,null,15,null,300


In [8]:
# 专门函数有'str'和'dt'名称空间

predicate = pl.col("fruits").str.contains("^b.*")

(df.select([
    predicate
]))

fruits
bool
true
true
false
false
true


In [9]:
# 利用谓语（predicate）进行过滤

df.filter(predicate)

A,fruits,B,cars,optional
i64,str,i64,str,i64
1,"""banana""",5,"""beetle""",28
2,"""banana""",4,"""audi""",300
5,"""banana""",1,"""beetle""",-30


In [10]:
# 谓语（predicate） 表达式可以被用于过滤

(df.select([
    pl.col("A").filter(pl.col("fruits").str.contains("^b.*")).sum(),
    (pl.col("B").filter(pl.col("cars").str.contains("^b.*")).sum() * pl.col("B").sum()).alias("some_compute()"), # .alias方法表示另起一列
]))

A,some_compute()
i64,i64
8,165


In [11]:
# 我们可以对列和（文字）值进行算术运算
# 可以在程序员不知道的情况下计算为1

some_var = 1

(df.select([
    ((pl.col("A") / 124.0 * pl.col("B")) / pl.sum("B") * some_var).alias("computed")
]))

computed
f64
0.002688
0.004301
0.004839
0.004301
0.002688


In [12]:
# 我们可以通过谓词（predicate）来组合列

(df.select([
    "fruits",
    "B",
    pl.when(pl.col("fruits") == "banana").then(pl.col("B")).otherwise(-1).alias("b")
]))

fruits,B,b
str,i64,i64
"""banana""",5,5
"""banana""",4,4
"""apple""",3,-1
"""apple""",2,-1
"""banana""",1,1


In [13]:
# 我们可以在列这一层通过折叠操作组合列

(df.select([
    "A",
    "B",
    pl.fold(0, lambda a, b: a + b, [pl.col("A"), "B", pl.col("B")**2, pl.col("A") / 2.0]).alias("fold")
]))

A,B,fold
i64,i64,f64
1,5,31.5
2,4,23.0
3,3,16.5
4,2,12.0
5,1,9.5


# 聚合上下文
* 表达式应用于组而不是列

In [14]:
# 我们仍可以组合很多表达式

(df.sort("cars").group_by("fruits")
    .agg([
        pl.col("B").sum().alias("B_sum"),
        pl.sum("B").alias("B_sum2"),  # 第一个是语法糖
        pl.first("fruits").alias("fruits_first"),
        pl.count("A").alias("count"),
        pl.col("cars").reverse()
    ]))

fruits,B_sum,B_sum2,fruits_first,count,cars
str,i64,i64,str,u32,list[str]
"""banana""",10,10,"""banana""",3,"[""beetle"", ""beetle"", ""audi""]"
"""apple""",5,5,"""apple""",2,"[""beetle"", ""beetle""]"


In [15]:
# 我们可以从“汽车”列下手

(df.sort("cars").group_by("fruits")
    .agg([
        pl.col("B").sum().alias("B_sum"),
        pl.sum("B").alias("B_sum2"),  # 第一个是语法糖
        pl.first("fruits").alias("fruits_first"),
        pl.count("A").alias("count"),
        pl.col("cars").reverse()
    ])).explode("cars")

fruits,B_sum,B_sum2,fruits_first,count,cars
str,i64,i64,str,u32,str
"""apple""",5,5,"""apple""",2,"""beetle"""
"""apple""",5,5,"""apple""",2,"""beetle"""
"""banana""",10,10,"""banana""",3,"""beetle"""
"""banana""",10,10,"""banana""",3,"""beetle"""
"""banana""",10,10,"""banana""",3,"""audi"""


In [16]:
(df.group_by("fruits")
    .agg([
        pl.col("B").sum().alias("B_sum"),
        pl.sum("B").alias("B_sum2"),  # 第一个是语法糖
        pl.first("fruits").alias("fruits_first"),
        pl.count(),
        pl.col("B").shift().alias("B_shifted")
    ])
 .explode("B_shifted")
)

fruits,B_sum,B_sum2,fruits_first,count,B_shifted
str,i64,i64,str,u32,i64
"""banana""",10,10,"""banana""",3,null
"""banana""",10,10,"""banana""",3,5
"""banana""",10,10,"""banana""",3,4
"""apple""",5,5,"""apple""",2,null
"""apple""",5,5,"""apple""",2,3


In [17]:
# 我们可以从“汽车”列下手

(df.sort("cars").group_by("fruits")
    .agg([
        pl.col("B").sum(),
        pl.sum("B").alias("B_sum2"),  # 第一个是语法糖
        pl.first("fruits").alias("fruits_first"),
        pl.count("A").alias("count"),
        pl.col("cars").reverse()
    ])).explode("cars")

fruits,B,B_sum2,fruits_first,count,cars
str,i64,i64,str,u32,str
"""apple""",5,5,"""apple""",2,"""beetle"""
"""apple""",5,5,"""apple""",2,"""beetle"""
"""banana""",10,10,"""banana""",3,"""beetle"""
"""banana""",10,10,"""banana""",3,"""beetle"""
"""banana""",10,10,"""banana""",3,"""audi"""


In [18]:
# 我们也可以得到一列组

(df.group_by("fruits")
    .agg([
         pl.col("B").shift().alias("shift_B"),
         pl.col("B").reverse().alias("rev_B"),
    ]))

fruits,shift_B,rev_B
str,list[i64],list[i64]
"""apple""","[null, 3]","[2, 3]"
"""banana""","[null, 5, 4]","[1, 4, 5]"


In [19]:
# 我们也可以在groupby中使用谓词

(df.group_by("fruits")
    .agg([
        pl.col("B").filter(pl.col("B") > 1).implode().keep_name(),
    ]))

fruits,B
str,list[list[i64]]
"""banana""","[[5, 4]]"
"""apple""","[[3, 2]]"


In [20]:
# 并且只根据谓词为真的值求和

(df.group_by("fruits")
    .agg([
        pl.col("B").filter(pl.col("B") > 1).mean(),
    ]))

fruits,B
str,f64
"""banana""",4.5
"""apple""",2.5


# 窗口函数!

* 用“超能力表”达。
* 选择上下文中的聚合


```python
pl.col("foo").aggregation_expression(..).over("column_used_to_group")
```


In [21]:
# groupby 2个不同的列

(df.select([
    "fruits",
    "cars",
    "B",
    pl.col("B").sum().over("fruits").alias("B_sum_by_fruits"),
    pl.col("B").sum().over("cars").alias("B_sum_by_cars"),
]))

fruits,cars,B,B_sum_by_fruits,B_sum_by_cars
str,str,i64,i64,i64
"""banana""","""beetle""",5,10,11
"""banana""","""audi""",4,10,4
"""apple""","""beetle""",3,5,11
"""apple""","""beetle""",2,5,11
"""banana""","""beetle""",1,10,11


In [22]:
# 按组反转B，并在原始df中显示结果

(df.select([
    "fruits",
    "B",
    pl.col("B").reverse().over("fruits").alias("B_reversed_by_fruits")
]))

fruits,B,B_reversed_by_fruits
str,i64,i64
"""banana""",5,1
"""banana""",4,4
"""apple""",3,2
"""apple""",2,3
"""banana""",1,5


In [23]:
# 将一列置于“fruits”中

(df.select([
    "fruits",
    "B",
    pl.col("B").shift().over("fruits").alias("lag_B_by_fruits")
]))


fruits,B,lag_B_by_fruits
str,i64,i64
"""banana""",5,null
"""banana""",4,5
"""apple""",3,null
"""apple""",2,3
"""banana""",1,4
